# World of Quantum Hands-on: QisK-I-T

This Notebook is divided into 2 parts:
- Code for error suppresion & dynamic decopuling Research Question (RQ1)
- Code for automated supervised learning best configuration Prediction in Optimization stage (RQ2)

    - Data-Preparation & Pass Selection and Experimentation
    - Model-Training and Experimentation
    - Evaluation

## Technical: Before you begin
- `conda create -n [env_name]`
- `conda activate [env_name]`
- `pip install -r ./requirements.txt`
- `conda update -all`

## Imports

In [1]:
import ast
import numpy as np
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, hamming_loss
from sklearn.metrics import f1_score
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import joblib
from pathlib import Path
from datetime import datetime



/opt/anaconda3/envs/Notebook/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Code for error suppresion & dynamic decopuling Research Question (RQ1)

In [2]:
# PLACEHOLDER RINORS AND JOSHUAS CODE

# 2. Code for automated supervised learning best configuration Prediction in Optimization stage (RQ2)

Below is the visualized pipeline

![My diagram](Resources/graph.png)

## Data-Preparation & Pass Selection and Experimentation

### hier der hackaton.py etc.

## Model-Training and Experimentation

### Data preprocessing

We do following steps:

1. a
2. b
3. c

In [3]:
dataset = pd.read_csv('data/training_data.csv', header=None) # TODO: remove header=NONE in next csv version

dataset.rename(
    columns={
      dataset.columns[0]:  "ID",
      dataset.columns[-1]: "configuration"
    },
    inplace=True
)

dataset.head()


,ID,1,2,3,4,5,6,7,8,configuration
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [4]:
df_configurations_with_id_with_duplicates = dataset[['ID', 'configuration']].copy()
df_features_with_id = dataset.drop(['configuration'], axis=1).copy()

In [5]:
df_configurations_with_id_with_duplicates.head()

,ID,configuration
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [6]:
df_features_with_id.head()

,ID,1,2,3,4,5,6,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0


In [7]:
df_configurations_with_id_with_duplicates['config_arr'] = (
    df_configurations_with_id_with_duplicates['configuration']
      .apply(ast.literal_eval)          # "[1,0,1,...]" → [1,0,1,...]
      #.apply(lambda lst: np.array(lst, dtype=int))
)
#df_ored_configuration = (
#    df_configurations_with_id_with_duplicates
#      .groupby('ID')['config_arr']
#      .agg(lambda arrs: np.bitwise_or.reduce(arrs.tolist()))
#      .reset_index()
#      .rename(columns={'config_arr':'label_vec'})
#)

df_configurations = df_configurations_with_id_with_duplicates[['ID','config_arr']].copy()

In [8]:
df_configurations.head()

,ID,config_arr
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [9]:
df_features_with_id.head()

,ID,1,2,3,4,5,6,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0


In [10]:
single_value_cols = df_features_with_id.columns[df_features_with_id.nunique(dropna=False) == 1]
single_value_cols

Index([6], dtype='object')

In [11]:
df_feat = df_features_with_id.drop(single_value_cols, axis=1).copy()
df_feat.head()

,ID,1,2,3,4,5,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0


In [12]:
df_feat.drop(['ID'], inplace=True, axis=1)
df_configurations.drop(['ID'], inplace=True, axis=1)

In [13]:
df_feat

,1,2,3,4,5,7,8
0,10,183,20,375,90,90,0
1,10,183,20,375,90,90,0
2,10,183,20,375,90,90,0
3,10,183,20,375,90,90,0
4,10,183,20,375,90,90,0
...,...,...,...,...,...,...,...
2165,9,54,18,297,108,108,0
2166,9,54,18,297,108,108,0
2167,9,54,18,297,108,108,0
2168,9,54,18,297,108,108,0


In [14]:
df_configurations

,config_arr
0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...
2165,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2166,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2167,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2168,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
df_feat.dtypes

1    int64
2    int64
3    int64
4    int64
5    int64
7    int64
8    int64
dtype: object

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_feat)
df_scaled = pd.DataFrame(X_scaled, columns=df_feat.columns, index=df_feat.index)
df_scaled

,1,2,3,4,5,7,8
0,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
1,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
2,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
3,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
4,-0.478707,-0.064169,-0.483159,-0.213629,-0.240287,-0.240335,-0.219793
...,...,...,...,...,...,...,...
2165,-0.599078,-0.163858,-0.603326,-0.242676,-0.222733,-0.222782,-0.219793
2166,-0.599078,-0.163858,-0.603326,-0.242676,-0.222733,-0.222782,-0.219793
2167,-0.599078,-0.163858,-0.603326,-0.242676,-0.222733,-0.222782,-0.219793
2168,-0.599078,-0.163858,-0.603326,-0.242676,-0.222733,-0.222782,-0.219793


In [17]:
y = np.vstack(df_configurations['config_arr'].values)
X = df_scaled.values

In [18]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, random_state=43
)

### Configuration for cross-validation of hyperparameters and of the model to gain reliable insights

In [19]:
# Configuration
DO_HPO          = True # hyperparameter search
HPO_NUM_TRIALS  = 60
N_OUTER_FOLDS   = 10
N_INNER_FOLDS   = 5
RANDOM_STATE    = 43

BASELINE_PARAMS = {
    "n_estimators": 1000,
}

In [20]:
outer_cv = MultilabelStratifiedKFold(
    n_splits=N_OUTER_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

inner_cv = MultilabelStratifiedKFold(
    n_splits=N_INNER_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

In [21]:
if DO_HPO:
    def objective(trial):
        param = {
            "n_estimators":       trial.suggest_int("n_estimators", 50, 300),
            "max_depth":          trial.suggest_int("max_depth", 3, 10),
            "learning_rate":      trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "subsample":          trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree":   trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "objective":          "binary:logistic",
            "tree_method":        "hist",
            "base_score":          0.5,
            "eval_metric":        "logloss",
            "random_state":       RANDOM_STATE
        }

        classifier = XGBClassifier(
        **param,
        )

        multiLabelClassifier = MultiOutputClassifier(classifier, n_jobs=-1)

        scores = cross_val_score(
            multiLabelClassifier,
            X_trainval,
            y_trainval,
            cv=inner_cv,
            scoring=make_scorer(f1_score, average='micro', zero_division=0),
            n_jobs=-1
        )
        # return the mean macro-F1 across folds
        return scores.mean()

    # 4) create & run the study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=HPO_NUM_TRIALS)

    # 5) extract best params
    best_params = study.best_params
    print("Best hyperparameters:", best_params)
else:
    best_params = BASELINE_PARAMS.copy()

[I 2025-06-25 12:47:26,479] A new study created in memory with name: no-name-9a383834-4d6d-4f52-a03a-ee578b4d7adf
[I 2025-06-25 12:47:27,699] Trial 0 finished with value: 0.6543284808123981 and parameters: {'n_estimators': 93, 'max_depth': 7, 'learning_rate': 0.016649295259629143, 'subsample': 0.6359008580988974, 'colsample_bytree': 0.6086958656786066}. Best is trial 0 with value: 0.6543284808123981.
[I 2025-06-25 12:47:28,669] Trial 1 finished with value: 0.658502314184837 and parameters: {'n_estimators': 144, 'max_depth': 8, 'learning_rate': 0.009408573206524218, 'subsample': 0.8009259815899048, 'colsample_bytree': 0.5260544640358544}. Best is trial 1 with value: 0.658502314184837.
[I 2025-06-25 12:47:29,787] Trial 2 finished with value: 0.5514287380619651 and parameters: {'n_estimators': 285, 'max_depth': 4, 'learning_rate': 0.0027991327880176574, 'subsample': 0.5700257816098986, 'colsample_bytree': 0.8393019769077879}. Best is trial 1 with value: 0.658502314184837.
[I 2025-06-25 12

Best hyperparameters: {'n_estimators': 169, 'max_depth': 10, 'learning_rate': 0.08481525362913016, 'subsample': 0.7000306392764534, 'colsample_bytree': 0.9498041316815203}


In [22]:
base_clf = XGBClassifier(
    **best_params,
    objective='binary:logistic',
    random_state=RANDOM_STATE,
    tree_method='hist',
    base_score=0.5,
    eval_metric='logloss'
)

multiLabelClassifier = MultiOutputClassifier(base_clf, n_jobs=-1)
scoring = {
    "f1_micro": make_scorer(f1_score, average='micro', zero_division=0),
    "f1_macro": make_scorer(f1_score, average='macro', zero_division=0),
    "hamming": make_scorer(hamming_loss)
}

cv_results = cross_validate(
    multiLabelClassifier, X_trainval, y_trainval,
    cv=outer_cv,
    scoring=scoring,
    return_train_score=False,
    n_jobs=-1
)

print("Outer-fold scores:")
for name in scoring:
    mean = cv_results[f"test_{name}"].mean()
    std  = cv_results[f"test_{name}"].std()
    print(f"  {name:>10s}: {mean:7.3f} ± {std:.3f}")

Outer-fold scores:
    f1_micro:   0.679 ± 0.009
    f1_macro:   0.253 ± 0.005
     hamming:   0.078 ± 0.002


## Evaluation Metrics

### F1-micro

* **Measures:** Overall accuracy by pooling all label decisions.
* **Formula:**

  $$
  \frac{2 \cdot \text{TP}_{\text{total}}}
       {2 \cdot \text{TP}_{\text{total}} + \text{FP}_{\text{total}} + \text{FN}_{\text{total}}}
  $$

---

### F1-macro

* **Measures:** Average F1 across labels, treating each label equally.
* **Formula:**

  $$
  \frac{1}{L}\sum_{i=1}^{L}
  \frac{2 \cdot \text{TP}_i}
       {2 \cdot \text{TP}_i + \text{FP}_i + \text{FN}_i}
  $$

---

### Hamming Loss

* **Measures:** Fraction of incorrect label decisions.
* **Formula:**

  $$
  \frac{\text{FP}_{\text{total}} + \text{FN}_{\text{total}}}
       {(\text{number of labels}) \times (\text{number of samples})}
  $$

Interpretation of final test-set metrics:
 F1-micro: 0.691
 F1-macro: 0.260
 Hamming : 0.075

- Micro-F1 & Hamming: We are getting ~68 % of bits right and misclassifying only 7–8 % on average
- Macro-F1: 0.26 is low, indicating we struggle on rare passes. The reason is possibly unsufficient data preprocessing, meaning underrepresenting rare instances, due to the limited time. This could be possibly gratly improved by better preprocessing / data weighting with custom weight functions.


In [23]:
multiLabelClassifier.fit(X_trainval, y_trainval)
y_pred = multiLabelClassifier.predict(X_test)

# Final test-set metrics, with zero_division=0 and formatted
f1_micro_test = f1_score(y_test, y_pred, average='micro', zero_division=0)
f1_macro_test = f1_score(y_test, y_pred, average='macro', zero_division=0)
hamming_test  = hamming_loss(y_test, y_pred)

print("\nFinal test-set metrics:")
print(f" F1-micro: {f1_micro_test:.3f}")
print(f" F1-macro: {f1_macro_test:.3f}")
print(f" Hamming : {hamming_test:.3f}")


Final test-set metrics:
 F1-micro: 0.694
 F1-macro: 0.263
 Hamming : 0.076


In [24]:
multiLabelClassifier.fit(X, y)

,estimator,"XGBClassifier...ree=None, ...)"
,n_jobs,-1
,objective,'binary:logistic'
,base_score,0.5
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9498041316815203
,device,None
,early_stopping_rounds,None


In [25]:
data_dir = Path("models")
data_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

model_filename  = f"qiskit_pass_predictor_{timestamp}.joblib"

model_path  = data_dir / model_filename


joblib.dump(multiLabelClassifier, model_path)
print(f"Model saved to {model_path}")

Model saved to models/qiskit_pass_predictor_20250625_124800.joblib


## Evaluation
We use the trained model to predict the optimal configuration for new circuits.
Then, we evaluate performance based on the number of qubits in the transpiled output, comparing it against Qiskit's default PassManager strategies: fast, normal, and slow.

In [26]:
# TODO: Add evaluation code against default optimization routine